In [1]:
import pandas as pd
from selenium import webdriver
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
from time import sleep
import json
import multiprocessing as mp
from multiprocessing import freeze_support

In [2]:
# basic params
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

header = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

crawler_shinhan = 'C:/Users/shic/Desktop/crawler_project/'
crawler_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/crawler_project/'

In [3]:
region_data = pd.read_csv(crawler_seunghun + 'data_save/kb_land/region_code.csv',
                          dtype = {'big_region_code' : str,
                                   'big_region_name' : str,
                                   'small_region_code' : str,
                                   'small_region_name' : str})
region_data = region_data[region_data['small_region_name']!='(전체)']

In [4]:
# make date list([year,month])
date_list = []
for year in range(2012,2022):
    year = str(year)
    for month in range(1,13):
        if int(month) < 10:
            month = '0' + str(month)
        else:
            month = str(month)

        if year == '2021' and month == '07':
            break

        date_list.append([year,month])

In [7]:
base_url = 'https://onland.kbstar.com/quics?page=C060737&%EB%B2%95%EC%A0%95%EB%8F%99%EB%8C%80%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C={}&%EB%B2%95%EC%A0%95%EB%8F%99%EC%A4%91%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C={}&%EA%B8%B0%EC%A4%80%EB%85%84={}&%EA%B8%B0%EC%A4%80%EC%9B%94={}&QSL=F'

In [9]:
region_data

,big_region_code,big_region_name,small_region_code,small_region_name
2,1100000000,서울,1199000000,서울강북
3,1100000000,서울,1130500000,서울강북구
4,1100000000,서울,1121500000,서울광진구
5,1100000000,서울,1135000000,서울노원구
6,1100000000,서울,1132000000,서울도봉구
...,...,...,...,...
178,4800000000,경남,4833000000,양산
179,4800000000,경남,4831000000,거제
180,4800000000,경남,4817000000,진주
181,4800000000,경남,4825000000,김해


In [158]:
def kb_land_crawler(dates):
    year  = dates[0]
    month = dates[1]

    df_list = []
    for region in region_data.iterrows():
        tmp_url = base_url.format(region[1]['big_region_code'],  # 대지역코드
                                  region[1]['small_region_code'],  # 중지역코드
                                  year, month)

        # URL 접속
        tmp_req = requests.get(tmp_url, headers=header)

        date_jisu_list = []
        data_jisu_list = []
        
        parse_jisu = re.search("var graphYn1 = [$](.+?);", tmp_req.text, re.S)
        if parse_jisu is None:
            date_jisu_list.append(0)
            data_jisu_list.append(0)
        else:
            parse_jisu = re.search("var graphYn1 = [$](.+?);", tmp_req.text, re.S)[0]
            parse_jisu = re.findall("\[(.*?)\]",parse_jisu)

            for parsed in parse_jisu:
                part_jisu = parsed.split('), ')

                jisu_date_parser = part_jisu[0].replace('Date.UTC(','').split(', ')

                date_jisu_list.append(','.join([jisu_date_parser[0],
                                                str(int(jisu_date_parser[1]) + 1),
                                                jisu_date_parser[2]]
                                     ))
                data_jisu_list.append(part_jisu[1])
                
        date_rate_list = []
        data_rate_list = []

        parse_rate = re.search('var graphYn2 = [$](.+?);', tmp_req.text, re.S)
        if parse_rate is None:
            date_rate_list.append(0)
            date_rate_list.append(0)
        else:
            parse_rate = re.search('var graphYn2 = [$](.+?);', tmp_req.text, re.S)[0]
            parse_rate = re.findall("\[(.*?)\]",parse_rate[0])

            for parsed in parse_rate:
                part_rate = parsed.split('), ')

                rate_date_parser = part_rate[0].replace('Date.UTC(','').split(', ')

                date_rate_list.append(','.join([rate_date_parser[0],
                                                str(int(rate_date_parser[1]) + 1),
                                                rate_date_parser[2]]
                                    ))
                data_rate_list.append(part_rate[1])

        #df_list.append(return_dataframe)
    return data_jisu_list